Прочтём данные из файлов с результатами A/B-тестирования:

In [1]:
import pandas as pd

In [2]:
orders = pd.read_csv('datasets/orders.csv', parse_dates=['date'])

In [3]:
visitors = pd.read_csv('datasets/visitors.csv', parse_dates=['date'])

In [4]:
orders

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B
...,...,...,...,...,...
1192,2662137336,3733762160,2019-08-14,6490,B
1193,2203539145,370388673,2019-08-14,3190,A
1194,1807773912,573423106,2019-08-14,10550,A
1195,1947021204,1614305549,2019-08-14,100,A


In [5]:
visitors

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
...,...,...,...
57,2019-08-27,B,720
58,2019-08-28,B,654
59,2019-08-29,B,531
60,2019-08-30,B,490


In [6]:
from solver.ab_reporter import ABReporter

In [7]:
example = ABReporter('datasets/visitors.csv', 'datasets/orders.csv')

In [8]:
cumulated = example.grouped_summary()

In [9]:
cumulated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           62 non-null     datetime64[ns]
 1   group          62 non-null     object        
 2   orders         62 non-null     int64         
 3   buyers         62 non-null     int64         
 4   revenue        62 non-null     int64         
 5   revenue_cm     62 non-null     int64         
 6   orders_cm      62 non-null     int64         
 7   buyers_cm      62 non-null     int64         
 8   visitors       62 non-null     int64         
 9   visitors_cm    62 non-null     int64         
 10  conversion_cm  62 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(8), object(1)
memory usage: 5.8+ KB


In [10]:
df = example.mannwhitneyu()

In [11]:
df

,0,1
0,Конверсия по сырым,0.016792
1,Средний чек по сырым,0.729291
2,Конверсия по очищенным,0.013043
3,Средний чек по очищенным,0.738482


In [ ]:
visitors_a = visitors.query('group == "A"').groupby(['date'], as_index=False).agg({'date' : 'max',
                                                                     'group' : 'max',
                                                                     'visitors' : 'sum'}, axis=1)
visitors_a['visitors'] = visitors_a['visitors'].cumsum()

In [ ]:
visitors_a

In [ ]:
orders_aggregated = orders.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                          'group' : 'max',
                                                                          'transactionId' : 'nunique',
                                                                          'visitorId' : 'nunique',
                                                                          'revenue' : 'sum'}, axis=1)

In [ ]:
visitors_a

In [ ]:
visitors_a_aggregated = visitors.query('group == "A"').groupby(['date'], as_index=False).agg({'date' : 'max',
                                                                                             'group' : 'max',
                                                                                             'visitors' : 'sum'}, axis=1)

In [ ]:
def cumulate_column(df, column):
    grouped_by_a = df[df['group'] == 'A'][column].cumsum()
    grouped_by_b = df[df['group'] == 'B'][column].cumsum()
    cumulated = pd.concat([grouped_by_a, grouped_by_b]).reset_index()
    cumulated.sort_values(by='index', inplace=True)
    return cumulated.set_index('index')

In [ ]:
visitors_aggregated = visitors.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                              'group' : 'max',
                                                                              'visitors' : 'sum'}, axis=1)

In [ ]:
orders_aggregated = orders.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                          'group' : 'max',
                                                                          'transactionId' : 'nunique',
                                                                          'visitorId' : 'nunique',
                                                                          'revenue' : 'sum'}, axis=1)

In [ ]:
visitors_aggregated

In [ ]:
orders_aggregated

In [ ]:
cumulate_column(visitors_aggregated, 'visitors')

In [ ]:
cumulate_column(orders_aggregated, 'revenue')

In [ ]:
visitors_aggregated['visitors'] = cumulate_column(visitors_aggregated, 'visitors')
orders_aggregated['transactionId'] = cumulate_column(orders_aggregated, 'transactionId')
orders_aggregated['visitorId'] = cumulate_column(orders_aggregated, 'visitorId')

In [ ]:
visitors_aggregated

In [ ]:
orders_aggregated

In [ ]:
cumulativeData = orders_aggregated.merge(visitors_aggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulativeData.head(11)

In [ ]:
cumulativeData = orders_aggregated.merge(visitors_aggregated)
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulativeData

In [ ]:
visitors[visitors['date'] == '2019-08-05']

In [ ]:
orders[orders['date'] == '2019-08-05'].count()

In [ ]:
temp = orders[orders['date'] == '2019-08-05']

In [ ]:
temp.groupby(['date'], as_index=False).agg({'date' : 'max',
                                            'group' : 'max',
                                            'transactionId' : 'nunique',
                                            'visitorId' : 'nunique',
                                            'revenue' : 'sum'}, axis=1)

In [ ]:
import matplotlib.pyplot as plt

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

# Строим график выручки группы А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='A')

# Строим график выручки группы B
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='B')

plt.legend()

In [ ]:
from solver.ab_reporter import ABReporter

In [ ]:
example = ABReporter('datasets/visitors.csv', 'datasets/orders.csv')

In [ ]:
cumulated = example.grouped_summary()

In [ ]:
cumulated.info()

In [ ]:
example.mannwhitneyu()

In [ ]:
    cumulated_copy = cumulated.copy()
    # cumulated_copy['date'] = cumulated_copy['date'].dt.date

In [ ]:
cumulated_copy['date'] = cumulated_copy['date'].dt.date

In [ ]:
cumulated_copy.info()

In [ ]:
def plot_cumulative_metrics():
    """Функция для визуализации кумулятивных метрик."""
    plt.figure(figsize=(25, 10))
    plt.style.use('seaborn-darkgrid')
    
    columns_to_pick = ['date','revenue', 'orders', 'conversion']
    cumulated_copy = cumulated.copy()
    cumulated_copy['date'] = cumulated_copy['date'].dt.date
    revenue_a = cumulated_copy.query('group == "A"')[columns_to_pick]
    revenue_b = cumulated_copy.query('group == "B"')[columns_to_pick]
    merged_revenues = revenue_a.merge(revenue_b, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
    

    # кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 3, 1)
    plt.plot(revenue_a['date'], revenue_a['revenue'], label='группа A')
    plt.plot(revenue_b['date'], revenue_b['revenue'], label='группа B')
    ax1.set_xticks(revenue_a['date'][::7])
    ax1.set_xticklabels(revenue_a['date'][::7])
    plt.legend()
    plt.ylabel('Выручка')
    plt.xlabel('Лайфтайм')
    plt.title('Графики кумулятивной выручки по дням и группам')
    
    ax2 = plt.subplot(2, 3, 2, sharex=ax1)    
    plt.plot(revenue_a['date'], revenue_a['revenue']/revenue_a['orders'], label='A')
    plt.plot(revenue_b['date'], revenue_b['revenue']/revenue_b['orders'], label='B')
    plt.legend()
    plt.ylabel('Средняя сумма чека')
    plt.xlabel('Лайфтайм')
    plt.title('Графики среднего чека по группам')
              
    ax3 = plt.subplot(2, 3, 3, sharex=ax1)   
    plt.plot(merged_revenues['date'], (merged_revenues['revenueB']/merged_revenues['ordersB'])/(merged_revenues['revenueA']/merged_revenues['ordersA'])-1)
    plt.axhline(y=0, color='black', linestyle='--')
    plt.ylabel('Отношение средних чеков')
    plt.xlabel('Лайфтайм')
    plt.title('График относительного различия для среднего чека')


    ax4 = plt.subplot(2, 3, 4, sharex=ax1)
    plt.plot(revenue_a['date'], revenue_a['conversion'], label='A')
    plt.plot(revenue_b['date'], revenue_b['conversion'], label='B')
    plt.legend()
    plt.ylabel('Конверсия')
    plt.xlabel('Лайфтайм')
    plt.title('График кумулятивной конверсии')
    
    ax5 = plt.subplot(2, 3, 5, sharex=ax1)
    plt.plot(merged_revenues['date'], merged_revenues['conversionB']/merged_revenues['conversionA']-1)
    plt.axhline(y=0, color='black', linestyle='--')
    plt.axhline(y=-0.1, color='grey', linestyle='--')
    plt.title('Относительный прирост конверсии группы B относительно группы A')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cumulative_metrics()

In [ ]:
cumulated.groupby(['group', 'date'])['revenue'].sum().plot()

In [ ]:
example.plot_cumulative_metrics()

In [ ]:
columns_to_pick = ['date','revenue', 'orders']
conversion_hist = cumulated[columns_to_pick]

In [ ]:
cumulated

In [ ]:
cumulated[['revenue']].plot()

In [ ]:
columns = [name for name in conversion_hist.index.names
           if name not in ['date']]
filtered_data = conversion_hist.pivot_table(
    index='date')

In [ ]:
filtered_data

Аналогично получим агрегированные кумулятивные по дням данные о посетителях интернет-магазина:


In [ ]:
df = visitors.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                        'group' : 'max',
                                                        'visitors' : 'sum'}, axis=1).sort_values(by=['date','group'])

In [ ]:
orders.boxplot('revenue')
plt.xlimit=2000

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(25, 5))
x_values = pd.Series(range(0, len(orders['revenue'])))
ax1 = plt.subplot(1, 3, 1)
sns.histplot(data=orders, x='revenue')
ax1.axvline(orders['revenue'].median(),linestyle = '--', color = '#FF1493', label = 'median')
ax1.axvline(orders['revenue'].mean(),linestyle = '--', color = 'orange', label = 'mean')
ax1.axvline(orders['revenue'].quantile(0.1),linestyle = '--', color = 'yellow', label = '1%')
ax1.axvline(orders['revenue'].quantile(0.99),linestyle = '--', color = 'yellow', label = '99%')
plt.ylabel('Число пользователей')
plt.xlabel('Сумма')
plt.title('Распределение выручки')

ax2 = plt.subplot(1, 3, 2)
sns.scatterplot(ax=ax2, x=x_values, y=orders['revenue'], hue=orders['revenue'], size=orders['revenue'], sizes=(1, 200), linewidth=0, data=orders)
plt.legend()
plt.ylabel('Сумма выручки')
plt.xlabel('Пользователи')
plt.title('Диаграмма рассеивания для выручки')

ax = plt.subplot(1, 3, 3)
sns.boxplot(x=orders['revenue'])
plt.xlabel('Выручка')
plt.title('Диаграмма размаха для выручки')

In [ ]:
def plotter(data, column):
    plt.figure(figsize=(25, 5))

    ax1 = plt.subplot(1, 3, 1)
    sns.histplot(data=data, x=column)
    ax1.axvline(data[column].median(),linestyle = '--', color = '#FF1493', label = 'median')
    ax1.axvline(data[column].mean(),linestyle = '--', color = 'orange', label = 'mean')
    ax1.axvline(data[column].quantile(0.1),linestyle = '--', color = 'yellow', label = '1%')
    ax1.axvline(data[column].quantile(0.99),linestyle = '--', color = 'yellow', label = '99%')
    plt.ylabel('Число пользователей')
    plt.xlabel('Сумма')
    plt.title('Распределение выручки')

    ax2 = plt.subplot(1, 3, 2)
    sns.scatterplot(ax=ax2, x=x_values, y=data[column], hue=data[column], size=data[column], sizes=(1, 200), linewidth=0, data=data)
    plt.legend()
    plt.ylabel('Сумма выручки')
    plt.xlabel('Пользователи')
    plt.title('Диаграмма рассеивания для выручки')

    ax = plt.subplot(1, 3, 3)
    sns.boxplot(x=data[column])
    plt.xlabel('Выручка')
    plt.title('Диаграмма размаха для выручки')
    plt.tight_layout()
    plt.show()

In [ ]:
plotter(orders, 'revenue')

In [ ]:
plotter(orders.query('revenue < 700'), 'revenue')

In [ ]:
orders

In [ ]:
ordersByUsers = (
    orders.groupby('visitorId', as_index=False)
    .agg({'transactionId': 'nunique'})
)

ordersByUsers.columns = ['userId', 'orders']

print(ordersByUsers.sort_values(by='orders', ascending=False).head(10)) 

In [ ]:
ordersByUsers

In [ ]:
plotter(ordersByUsers, 'orders')

In [ ]:
import pandas as pd
import scipy.stats as stats
import datetime as dt
import numpy as np

visitors = pd.read_csv('datasets/visitors.csv')

orders = pd.read_csv('datasets/orders.csv')

visitorsADaily = visitors[visitors['group'] == 'A'][['date', 'visitors']]
visitorsADaily.columns = ['date', 'visitorsPerDateA']

visitorsACummulative = visitorsADaily.apply(
    lambda x: visitorsADaily[visitorsADaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateA': 'sum'}
    ),
    axis=1,
)
visitorsACummulative.columns = ['date', 'visitorsCummulativeA']

visitorsBDaily = visitors[visitors['group'] == 'B'][['date', 'visitors']]
visitorsBDaily.columns = ['date', 'visitorsPerDateB']

visitorsBCummulative = visitorsBDaily.apply(
    lambda x: visitorsBDaily[visitorsBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateB': 'sum'}
    ),
    axis=1,
)
visitorsBCummulative.columns = ['date', 'visitorsCummulativeB']

ordersADaily = (
    orders[orders['group'] == 'A'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)
ordersADaily.columns = ['date', 'ordersPerDateA', 'revenuePerDateA']

ordersACummulative = ordersADaily.apply(
    lambda x: ordersADaily[ordersADaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateA': 'sum', 'revenuePerDateA': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
ordersACummulative.columns = [
    'date',
    'ordersCummulativeA',
    'revenueCummulativeA',
]

ordersBDaily = (
    orders[orders['group'] == 'B'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)
ordersBDaily.columns = ['date', 'ordersPerDateB', 'revenuePerDateB']

ordersBCummulative = ordersBDaily.apply(
    lambda x: ordersBDaily[ordersBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateB': 'sum', 'revenuePerDateB': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
ordersBCummulative.columns = [
    'date',
    'ordersCummulativeB',
    'revenueCummulativeB',
]

data = (
    ordersADaily.merge(
        ordersBDaily, left_on='date', right_on='date', how='left'
    )
    .merge(ordersACummulative, left_on='date', right_on='date', how='left')
    .merge(ordersBCummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsADaily, left_on='date', right_on='date', how='left')
    .merge(visitorsBDaily, left_on='date', right_on='date', how='left')
    .merge(visitorsACummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsBCummulative, left_on='date', right_on='date', how='left')
)

print(data.head(5))

ordersByUsersA = (
    orders[orders['group'] == 'A']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersA.columns = ['userId', 'orders']

ordersByUsersB = (
    orders[orders['group'] == 'B']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersB.columns = ['userId', 'orders']

sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0,index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])),name='orders'),],axis=0,)

sampleB = pd.concat(
    [
        ordersByUsersB['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)
print("{0:.5f}".format(stats.mannwhitneyu(sampleA, sampleB)[1]))
print("{0:.3f}".format((data['ordersPerDateB'].sum()/data['visitorsPerDateB'].sum())/(data['ordersPerDateA'].sum()/data['visitorsPerDateA'].sum())-1))


In [ ]:
df

In [ ]:
sampleA

In [ ]:
data

In [ ]:
cumulated

In [ ]:
ordersByUsersA = (
    orders[orders['group'] == 'A']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersA.columns = ['userId', 'orders']

ordersByUsersB = (
    orders[orders['group'] == 'B']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersB.columns = ['userId', 'orders']

In [ ]:
ordersByUsersA

In [ ]:
ordersByUsersB

In [ ]:
pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])))

In [ ]:
list_of_shit = pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])))

In [ ]:
pd.concat([ordersByUsersA['orders'], list_of_shit])[499]

In [ ]:
orders.query('group == "B"')

In [ ]:
np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders']))

In [ ]:
import pandas as pd
import scipy.stats as stats
import datetime as dt
import numpy as np

In [ ]:
sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0,index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])),name='orders'),],axis=0,)

sampleB = pd.concat([ordersByUsersB['orders'], pd.Series(0,index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), name='orders', ),], axis=0,
)

In [ ]:
orders.query('group == "A"').groupby('visitorId', as_index=False).agg({'transactionId': 'count'})

In [ ]:
def mannwhitneyu():
    columns = ['userId', 'orders']
    orders_by_a = (orders.query('group == "A"').groupby('visitorId', as_index=False).agg({'transactionId': pd.Series.nunique}))
    orders_by_b = (orders.query('group == "B"').groupby('visitorId', as_index=False).agg({'transactionId': pd.Series.nunique}))
    
    result = stats.mannwhitneyu(sampleA, sampleB)
    ratio = (data['ordersPerDateB'].sum()/data['visitorsPerDateB'].sum())/(data['ordersPerDateA'].sum()/data['visitorsPerDateA'].sum())-1
    print(f'{result[1]:.5f}')
    print(f'{ratio:.3f}')

In [ ]:
mannwhitneyu()

In [ ]:
sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0,index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])),name='orders'),],axis=0,)

sampleB = pd.concat([ordersByUsersB['orders'], pd.Series(0,index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), name='orders',),], axis=0,)
print("{0:.5f}".format(stats.mannwhitneyu(sampleA, sampleB)[1]))
print("{0:.3f}".format((data['ordersPerDateB'].sum()/data['visitorsPerDateB'].sum())/(data['ordersPerDateA'].sum()/data['visitorsPerDateA'].sum())-1))


In [ ]:
ordersByUsersA['orders']

In [ ]:
print('{0:.3f}'.format(stats.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'])[1]))
print('{0:.3f}'.format(orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1))

In [ ]:
ouliers_a = 

In [ ]:
usersWithManyOrders = pd.concat([ordersByUsersA[ordersByUsersA['orders'] > 2]['userId'], ordersByUsersB[ordersByUsersB['orders'] > 2]['userId'],],axis=0,)
usersWithExpensiveOrders = orders[orders['revenue'] > 28000]['visitorId']
abnormalUsers = (pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis=0).drop_duplicates().sort_values())

sampleAFiltered = pd.concat([ordersByUsersA[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))]['orders'], pd.Series(0,index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])),name='orders',),],axis=0,)

sampleBFiltered = pd.concat([ordersByUsersB[np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))]['orders'],pd.Series(0,index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])),name='orders',),],axis=0,)

print('{0:.5f}'.format(stats.mannwhitneyu(sampleAFiltered, sampleBFiltered)[1]))
print('{0:.3f}'.format(sampleBFiltered.mean()/sampleAFiltered.mean()-1))

In [ ]:
print(
    '{0:.3f}'.format(
        stats.mannwhitneyu(orders[np.logical_and(orders['group'] == 'A',np.logical_not(orders['visitorId'].isin(abnormalUsers)),)]['revenue'],orders[np.logical_and(orders['group'] == 'B',np.logical_not(orders['visitorId'].isin(abnormalUsers)),)]['revenue'],)[1]
    )
)

print(
    "{0:.3f}".format(
        orders[
            np.logical_and(
                orders['group'] == 'B',
                np.logical_not(orders['visitorId'].isin(abnormalUsers)),
            )
        ]['revenue'].mean()
        / orders[
            np.logical_and(
                orders['group'] == 'A',
                np.logical_not(orders['visitorId'].isin(abnormalUsers)),
            )
        ]['revenue'].mean()
        - 1
    )
)

In [ ]:
sampleAFiltered = ordersByUsersA[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))]['orders']
sampleBFiltered = ordersByUsersB[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))]['orders']

In [ ]:
usersWithManyOrders = pd.concat(
    [
        ordersByUsersA[ordersByUsersA['orders'] > 2]['userId'],
        ordersByUsersB[ordersByUsersB['orders'] > 2]['userId'],
    ],
    axis=0,
)
usersWithExpensiveOrders = orders[orders['revenue'] > 28000]['visitorId']
abnormalUsers = (
    pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis=0)
    .drop_duplicates()
    .sort_values()
)

sampleAFiltered = pd.concat(
    [
        ordersByUsersA[
            np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

sampleBFiltered = pd.concat(
    [
        ordersByUsersB[
            np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

print('{0:.5f}'.format(stats.mannwhitneyu(sampleAFiltered, sampleBFiltered)[1]))
print('{0:.3f}'.format(sampleBFiltered.mean()/sampleAFiltered.mean()-1))

In [ ]:
sampleAFiltered

In [ ]:
sampleBFiltered

In [ ]:
sampleAFiltered = ordersByUsersA[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))]['orders']
sampleBFiltered = ordersByUsersB[np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))]['orders']

In [ ]:
print('{0:.5f}'.format(stats.mannwhitneyu(sampleAFiltered, sampleBFiltered)[1]))
print('{0:.3f}'.format(sampleBFiltered.mean()/sampleAFiltered.mean()-1))

In [ ]:
reporter.orders

Посчитайте статистическую значимость различий в конверсии между группами по «сырым» данным. Сделайте выводы и предположения.
Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным. Сделайте выводы и предположения.
Посчитайте статистическую значимость различий в конверсии между группами по «очищенным» данным. Сделайте выводы и предположения.
Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным. Сделайте выводы и предположения.

In [ ]:
def get_placeholders(data):
    full_length = cumulated.query('group == "A"')['visitors'].sum()
    index = np.arange(full_length - len(data), name='orders')
    placeholders = pd.Series(0, index=index)
    return placeholders    

In [ ]:
def prepare_data_for_stat():
    orders_by_a = (orders.query('group == "A"').groupby('visitorId', as_index=False).agg({'transactionId': 'nunique'}))
    orders_by_b = (orders.query('group == "B"').groupby('visitorId', as_index=False).agg({'transactionId': 'nunique'}))
    
    sample_raw_a = pd.concat([orders_by_a['orders'], get_placeholders(orders_by_a)], axis=0)
    sample_raw_b = pd.concat([orders_by_b['orders'], get_placeholders(orders_by_b)], axis=0)
        
    outliers_orders_a = orders_by_a.query('orders > 2')['visitorId']
    outliers_orders_b = orders_by_b.query('orders > 2')['visitorId']
    outliers_orders = pd.concat([outliers_orders_a, outliers_orders_b], axis=0)
    
    outliers_revenue = order.query('revenue > 28000')['visitorId']
    
    anomalies = (pd.concat([outliers_orders, outliers_revenue], axis=0).drop_duplicates().sort_values())
    
    revenue_a = orders.query('group =="A"')['revenue']
    revenue_b = orders.query('group =="B"')['revenue']
    
    clean_revenue_a = orders.query('group == "A" and visitorId not in @anomalies')['revenue']
    clean_revenue_b = orders.query('group == "B" and visitorId not in @anomalies')['revenue']
    
    cleaned_by_a = orders_by_a.query('visitorId not in @anomalies')['orders']
    cleaned_by_b = orders_by_b.query('visitorId not in @anomalies')['orders']
    
    sample_clean_a = pd.concat([cleaned_by_a, get_placeholders(orders_by_a)])
    sample_clean_b = pd.concat([cleaned_by_b, get_placeholders(orders_by_b)])
    
    prepared_data = {'sample_raw_a': sample_raw_a,
                     'sample_raw_b': sample_raw_b,
                     'revenue_a': revenue_a,
                     'revenue_b': revenue_b,
                     'clean_revenue_a': clean_revenue_a,
                     'clean_revenue_b': clean_revenue_b,
                     'sample_clean_a': sample_clean_a,
                     'sample_clean_b': sample_clean_b}
    
    return prepared_data
    


In [ ]:
def mannwhitneyu():
    result = []
    
    # Подготовим данные
    prepared_data = prepare_data_for_stat()
    
    # Считаем статистическую значимость различий в конверсии между группами по «сырым» данным
    raw_conv_result = stats.mannwhitneyu(prepared_data['sample_raw_a'], prepared_data['sample_raw_b'])[1]
    result.append(raw_conv_result)
    
    # Считаем статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным
    raw_revenue_result = stats.mannwhitneyu(prepared_data['revenue_a'], prepared_data['revenue_b'])[1]
    result.append(raw_revenue_result)
    
    # Считаем статистическую значимость различий в конверсии между группами по «очищенным» данным
    clean_conv_result = stats.mannwhitneyu(prepared_data['sample_clean_a'], prepared_data['sample_clean_b'])[1]
    result.append(clean_conv_result)
    
    # Считаем статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным
    clean_revenue_result = stats.mannwhitneyu(prepared_data['clean_revenue_a'], prepared_data['clean_revenue_b'])[1]
    result.append(clean_revenue_result)

    result = pd.DataFrame(result)

In [ ]:
orders.query('group == "A" and visitorId not in @anomalies')['revenue']

In [ ]:
orders[np.logical_and(orders['group'] == 'A',np.logical_not(orders['visitorId'].isin(abnormalUsers)),)]['revenue'],orders[np.logical_and(orders['group'] == 'B',np.logical_not(orders['visitorId'].isin(abnormalUsers)),)

In [ ]:
def mannwhitneyu():
    result = []
    
    # Подготовим данные
    orders_by_a = (orders.query('group == "A"').groupby('visitorId', as_index=False).agg({'transactionId': 'nunique'}))
    orders_by_b = (orders.query('group == "B"').groupby('visitorId', as_index=False).agg({'transactionId': 'nunique'}))
    
    
    usersWithManyOrders = pd.concat([ordersByUsersA[ordersByUsersA['orders'] > 2]['userId'], ordersByUsersB[ordersByUsersB['orders'] > 2]['userId'],],axis=0,)
    usersWithExpensiveOrders = orders[orders['revenue'] > 28000]['visitorId']
    abnormalUsers = (pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis=0).drop_duplicates().sort_values())

    sampleAFiltered = pd.concat([ordersByUsersA[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))]['orders'], pd.Series(0,index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])),name='orders',),],axis=0,)

    sampleBFiltered = pd.concat([ordersByUsersB[np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))]['orders'],pd.Series(0,index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])),name='orders',),],axis=0,)
    
    # Считаем статистическую значимость различий в конверсии между группами по «сырым» данным
    sample_a = pd.concat([orders_by_a['orders'],pd.Series(0,index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])),name='orders'),],axis=0,)
    sample_b = pd.concat([orders_by_b['orders'], pd.Series(0,index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), name='orders',),], axis=0,)
    result.append(stats.mannwhitneyu(sampleA, sampleB))
    
    # Считаем статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным
    revenue_a = orders.query('group =="A"')['revenue']
    revenue_b = orders.query('group =="B"')['revenue']
    result.append(stats.mannwhitneyu(revenue_a, revenue_b))
    
    # Считаем статистическую значимость различий в конверсии между группами по «очищенным» данным
    stats.mannwhitneyu(sampleAFiltered, sampleBFiltered)[1]
    
    # Считаем статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным
    stats.mannwhitneyu(orders[np.logical_and(orders['group'] == 'A',np.logical_not(orders['visitorId'].isin(abnormalUsers)),)]['revenue'],orders[np.logical_and(orders['group'] == 'B',np.logical_not(orders['visitorId'].isin(abnormalUsers)),)]['revenue'],)[1]